In [1]:
import os
import cv2
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
# Paths to your images and ground truth folders
image_folder = "images/"
ground_truth_folder = "masks/"

In [3]:
# Function to calculate metrics for a given set of parameters
def evaluate_canny_parameters(image_files, ground_truth_files, num_images, low_threshold, high_threshold, blur_kernel):
    sum_precision = 0
    sum_recall = 0
    sum_f1 = 0

    for img_file, gt_file in zip(image_files[:num_images], ground_truth_files[:num_images]):
        # Load the image and ground truth
        img_path = os.path.join(image_folder, img_file)
        gt_path = os.path.join(ground_truth_folder, gt_file)

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        ground_truth = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

        # Apply Gaussian blur
        blurred_img = cv2.GaussianBlur(img, (blur_kernel, blur_kernel), 0)

        # Apply Canny edge detection
        edges = cv2.Canny(blurred_img, low_threshold, high_threshold)

        # Threshold ground truth to binary (0 or 255 -> 0 or 1)
        _, ground_truth_binary = cv2.threshold(ground_truth, 127, 1, cv2.THRESH_BINARY)

        # Flatten the arrays for metric calculation
        edges_binary = (edges > 0).astype(np.uint8).flatten()
        ground_truth_flat = ground_truth_binary.flatten()

        # Compute metrics for the current image
        precision = precision_score(ground_truth_flat, edges_binary, zero_division=0)
        recall = recall_score(ground_truth_flat, edges_binary, zero_division=0)
        f1 = f1_score(ground_truth_flat, edges_binary, zero_division=0)

        # Sum metrics
        sum_precision += precision
        sum_recall += recall
        sum_f1 += f1

    # Average metrics
    avg_precision = sum_precision / num_images
    avg_recall = sum_recall / num_images
    avg_f1 = sum_f1 / num_images

    return avg_precision, avg_recall, avg_f1



In [5]:
# List all image files
image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')])
ground_truth_files = sorted([f for f in os.listdir(ground_truth_folder) if f.endswith('.jpg')])

# Ensure both folders have the same number of files
if len(image_files) != len(ground_truth_files):
    raise ValueError("Number of images and ground truth files do not match!")

# User-defined number of images to process
num_images = 100

# Hyperparameter tuning for Canny edge detection and Gaussian blur
best_f1 = 0
best_params = (0, 0, 0)

for blur_kernel in range(1, 12, 2):  # Iterate over odd kernel sizes for Gaussian blur
    for low_threshold in range(50, 150, 10):
        for high_threshold in range(100, 250, 10):
            avg_precision, avg_recall, avg_f1 = evaluate_canny_parameters(
                image_files, ground_truth_files, num_images, low_threshold, high_threshold, blur_kernel
            )

            print(f"Blur: {blur_kernel}, Low: {low_threshold}, High: {high_threshold} -> Precision: {avg_precision:.4f}, Recall: {avg_recall:.4f}, F1: {avg_f1:.4f}")

            if avg_f1 > best_f1:
                best_f1 = avg_f1
                best_params = (blur_kernel, low_threshold, high_threshold)

# Print the best parameters and corresponding metrics
print(f"Best Parameters: Blur={best_params[0]}, Low={best_params[1]}, High={best_params[2]} with F1 Score={best_f1:.4f}")


Blur: 1, Low: 50, High: 100 -> Precision: 0.0365, Recall: 0.3675, F1: 0.0574
Blur: 1, Low: 50, High: 110 -> Precision: 0.0401, Recall: 0.3633, F1: 0.0614
Blur: 1, Low: 50, High: 120 -> Precision: 0.0437, Recall: 0.3591, F1: 0.0658
Blur: 1, Low: 50, High: 130 -> Precision: 0.0483, Recall: 0.3547, F1: 0.0712
Blur: 1, Low: 50, High: 140 -> Precision: 0.0531, Recall: 0.3489, F1: 0.0770
Blur: 1, Low: 50, High: 150 -> Precision: 0.0587, Recall: 0.3427, F1: 0.0834
Blur: 1, Low: 50, High: 160 -> Precision: 0.0651, Recall: 0.3358, F1: 0.0905
Blur: 1, Low: 50, High: 170 -> Precision: 0.0724, Recall: 0.3301, F1: 0.0980
Blur: 1, Low: 50, High: 180 -> Precision: 0.0809, Recall: 0.3216, F1: 0.1057
Blur: 1, Low: 50, High: 190 -> Precision: 0.0891, Recall: 0.3130, F1: 0.1126
Blur: 1, Low: 50, High: 200 -> Precision: 0.0978, Recall: 0.3037, F1: 0.1192
Blur: 1, Low: 50, High: 210 -> Precision: 0.1070, Recall: 0.2933, F1: 0.1252
Blur: 1, Low: 50, High: 220 -> Precision: 0.1148, Recall: 0.2812, F1: 0.1286

In [ ]:
# Best Parameters: Blur=3, Low=60, High=170
